A camada Silver contém dados tratados e padronizados, prontos para modelagem.
Principais transformações realizadas:

- Remover _rescued_data
- Padronizar tipos numéricos
- Criar colunas de ano e mês
- Criar faixa etária (pagador e recebedor)
- Padronizar textos (UPPER)
- Garantir consistência para análises

In [0]:
CREATE SCHEMA IF NOT EXISTS silver;
USE SCHEMA silver;

## Definição das faixas etárias

Para as faixas etárias irei utilizar:

| Faixa         | Intervalo        |
| ------------- | ---------------- |
| 0–17          | Menor de idade   |
| 18–29         | Jovem adulto     |
| 30–44         | Adulto           |
| 45–59         | Meia-idade       |
| 60+           | Idoso            |
| Não informado | Null ou inválido |


## Tratamento dos dados

Irei iniciar tratando os dados de idade, das colunas PAG_IDADE e REC_IDADE



Listar valores distintos (pagador)

In [0]:
SELECT DISTINCT PAG_IDADE
FROM bronze.transacoes_pix
ORDER BY PAG_IDADE;


PAG_IDADE
Nao informado
Nao se aplica
até 19 anos
entre 20 e 29 anos
entre 30 e 39 anos
entre 40 e 49 anos
entre 50 e 59 anos
mais de 60 anos


Tratamento da idade do pagador

In [0]:
SELECT
  PAG_IDADE AS pag_idade_original,

  CASE
    WHEN PAG_IDADE = 'até 19 anos' THEN '0-19'
    WHEN PAG_IDADE = 'entre 20 e 29 anos' THEN '20-29'
    WHEN PAG_IDADE = 'entre 30 e 39 anos' THEN '30-39'
    WHEN PAG_IDADE = 'entre 40 e 49 anos' THEN '40-49'
    WHEN PAG_IDADE = 'entre 50 e 59 anos' THEN '50-59'
    WHEN PAG_IDADE = 'mais de 60 anos' THEN '60+'
    WHEN PAG_IDADE = 'Nao se aplica' THEN 'Nao se aplica'
    ELSE 'NAO_INFORMADO'
  END AS pag_faixa_etaria,

  COUNT(*) AS total
FROM bronze.transacoes_pix
GROUP BY PAG_IDADE, pag_faixa_etaria
ORDER BY total DESC;


pag_idade_original,pag_faixa_etaria,total
Nao se aplica,Nao se aplica,104833
entre 30 e 39 anos,30-39,84181
entre 20 e 29 anos,20-29,83613
entre 40 e 49 anos,40-49,83184
entre 50 e 59 anos,50-59,80291
mais de 60 anos,60+,77334
até 19 anos,0-19,73544
Nao informado,NAO_INFORMADO,11944


Listar valores distintos (recebedor)

In [0]:
SELECT DISTINCT REC_IDADE
FROM bronze.transacoes_pix
ORDER BY REC_IDADE;


REC_IDADE
Nao informado
Nao se aplica
até 19 anos
entre 20 e 29 anos
entre 30 e 39 anos
entre 40 e 49 anos
entre 50 e 59 anos
mais de 60 anos


Tratamento da idade do recebedor

In [0]:
SELECT
  REC_IDADE AS rec_idade_original,

  CASE
    WHEN REC_IDADE = 'até 19 anos' THEN '0-19'
    WHEN REC_IDADE = 'entre 20 e 29 anos' THEN '20-29'
    WHEN REC_IDADE = 'entre 30 e 39 anos' THEN '30-39'
    WHEN REC_IDADE = 'entre 40 e 49 anos' THEN '40-49'
    WHEN REC_IDADE = 'entre 50 e 59 anos' THEN '50-59'
    WHEN REC_IDADE = 'mais de 60 anos' THEN '60+'
    WHEN REC_IDADE = 'Nao se aplica' THEN 'Nao se aplica'
    ELSE 'NAO_INFORMADO'
  END AS rec_faixa_etaria,

  COUNT(*) AS total
FROM bronze.transacoes_pix
GROUP BY REC_IDADE, rec_faixa_etaria
ORDER BY total DESC;


rec_idade_original,rec_faixa_etaria,total
Nao se aplica,Nao se aplica,144555
entre 30 e 39 anos,30-39,77082
entre 40 e 49 anos,40-49,76508
entre 20 e 29 anos,20-29,76218
entre 50 e 59 anos,50-59,74333
mais de 60 anos,60+,72235
até 19 anos,0-19,65607
Nao informado,NAO_INFORMADO,12386


Agora vou tratar a coluna valor e criar uma padronização para casa do milhão e converter para real brasileiro.

In [0]:
SELECT
  VALOR AS valor_string_original,

  -- Valor numérico (correto para cálculos)
  TRY_CAST(
    REPLACE(
      REPLACE(VALOR, '.', ''),
      ',', '.'
    ) AS DECIMAL(18,2)
  ) AS valor_reais,

  -- Valor formatado no padrão brasileiro 9.999.999,99
  CONCAT(
    regexp_replace(
      regexp_replace(
        FORMAT_NUMBER(
          TRY_CAST(
            REPLACE(
              REPLACE(VALOR, '.', ''),
              ',', '.'
            ) AS DECIMAL(18,2)
          ),
          2
        ),
        ',', '.'
      ),
      '\\.(?=[0-9]{2}$)', ','
    )
  ) AS valor_formatado_br
FROM bronze.transacoes_pix
LIMIT 10;


valor_string_original,valor_reais,valor_formatado_br
"7626929,67",7626929.67,"7.626.929,67"
"7525,59",7525.59,"7.525,59"
"255424,33",255424.33,"255.424,33"
"668315,98",668315.98,"668.315,98"
"1226112,51",1226112.51,"1.226.112,51"
"23848,56",23848.56,"23.848,56"
"58934,01",58934.01,"58.934,01"
"24371515,36",24371515.36,"24.371.515,36"
"70869,33",70869.33,"70.869,33"
"37606858,96",37606858.96,"37.606.858,96"


A formatação monetária no padrão brasileiro foi aplicada exclusivamente na camada de apresentação, utilizando expressões regulares para conversão controlada dos separadores numéricos, preservando o valor decimal original para cálculos.

Agora vou criar a Tabela Silver

In [0]:
CREATE OR REPLACE TABLE silver.transacoes_pix AS
SELECT
  -- Tempo
  AnoMes,
  CAST(SUBSTRING(AnoMes, 1, 4) AS INT) AS ano,
  CAST(SUBSTRING(AnoMes, 5, 2) AS INT) AS mes,

  -- Tipo de pessoa
  PAG_PFPJ AS pag_tipo_pessoa,
  REC_PFPJ AS rec_tipo_pessoa,

  -- Região
  PAG_REGIAO AS pag_regiao,
  REC_REGIAO AS rec_regiao,

  -- Faixa etária (pagador)
  CASE
    WHEN PAG_IDADE = 'até 19 anos' THEN '0-19'
    WHEN PAG_IDADE = 'entre 20 e 29 anos' THEN '20-29'
    WHEN PAG_IDADE = 'entre 30 e 39 anos' THEN '30-39'
    WHEN PAG_IDADE = 'entre 40 e 49 anos' THEN '40-49'
    WHEN PAG_IDADE = 'entre 50 e 59 anos' THEN '50-59'
    WHEN PAG_IDADE = 'mais de 60 anos' THEN '60+'
    WHEN PAG_IDADE = 'Nao se aplica' THEN 'Nao se aplica'
    ELSE 'NAO_INFORMADO'
  END AS pag_faixa_etaria,

  -- Faixa etária (recebedor)
  CASE
    WHEN REC_IDADE = 'até 19 anos' THEN '0-19'
    WHEN REC_IDADE = 'entre 20 e 29 anos' THEN '20-29'
    WHEN REC_IDADE = 'entre 30 e 39 anos' THEN '30-39'
    WHEN REC_IDADE = 'entre 40 e 49 anos' THEN '40-49'
    WHEN REC_IDADE = 'entre 50 e 59 anos' THEN '50-59'
    WHEN REC_IDADE = 'mais de 60 anos' THEN '60+'
    WHEN REC_IDADE = 'Nao se aplica' THEN 'Nao se aplica'
    ELSE 'NAO_INFORMADO'
  END AS rec_faixa_etaria,

  -- Valor numérico em reais (para cálculo)
  TRY_CAST(
    REPLACE(
      REPLACE(VALOR, '.', ''),
      ',', '.'
    ) AS DECIMAL(18,2)
  ) AS valor_reais,

  -- Valor formatado no padrão brasileiro (SEM R$)
  regexp_replace(
    regexp_replace(
      FORMAT_NUMBER(
        TRY_CAST(
          REPLACE(
            REPLACE(VALOR, '.', ''),
            ',', '.'
          ) AS DECIMAL(18,2)
        ),
        2
      ),
      ',', '.'
    ),
    '\\.(?=[0-9]{2}$)', ','
  ) AS valor_formatado_br,

  -- Quantidade
  CAST(QUANTIDADE AS BIGINT) AS quantidade_total

FROM bronze.transacoes_pix;


num_affected_rows,num_inserted_rows


Validação rápida

In [0]:
SELECT
  valor_reais,
  valor_formatado_br
FROM silver.transacoes_pix
LIMIT 10;


valor_reais,valor_formatado_br
7626929.67,"7.626.929,67"
7525.59,"7.525,59"
255424.33,"255.424,33"
668315.98,"668.315,98"
1226112.51,"1.226.112,51"
23848.56,"23.848,56"
58934.01,"58.934,01"
24371515.36,"24.371.515,36"
70869.33,"70.869,33"
37606858.96,"37.606.858,96"


Os valores financeiros foram armazenados na camada Silver exclusivamente em formato numérico decimal, garantindo precisão para cálculos e agregações. A formatação monetária no padrão brasileiro foi aplicada apenas na camada de apresentação, respeitando a separação entre armazenamento e visualização dos dados.

# Qualidade dos Dados

Antes da realização das análises exploratórias e das análises de negócio, foi conduzida uma etapa de avaliação da qualidade dos dados, com o objetivo de garantir a confiabilidade e a consistência das informações utilizadas ao longo do pipeline analítico desenvolvido neste trabalho. Essa etapa foi realizada predominantemente sobre a camada Silver, que concentra os dados já tratados e padronizados, sendo o ponto mais adequado para validação antes da modelagem dimensional.

A análise de qualidade teve como foco a verificação do volume de registros, a validação do schema e dos tipos de dados, a identificação de valores nulos, a avaliação dos domínios dos dados numéricos, a análise da distribuição dos valores e a detecção de possíveis duplicidades. Adicionalmente, foram realizadas verificações de integridade referencial visando assegurar a consistência entre os dados que compõem o modelo estrela na camada Gold.

Essas verificações permitem assegurar que os dados utilizados nas análises apresentam estrutura adequada, atributos coerentes com as regras de negócio e relações consistentes entre fatos e dimensões, reduzindo o risco de anomalias que poderiam comprometer a interpretação e a confiabilidade dos resultados obtidos.

## Volume de registros por tabela

A análise de volume de registros foi utilizada para verificar se a quantidade de dados permaneceu consistente ao longo do pipeline, garantindo que não houve perda ou duplicação indevida de registros durante as etapas de ingestão, tratamento e modelagem. Essa verificação é fundamental para assegurar que as análises realizadas na camada Gold representam fielmente o conjunto de dados originalmente disponibilizado pela fonte.

In [0]:
SELECT
  'silver.transacoes_pix' AS tabela,
  COUNT(*) AS total_registros
FROM silver.transacoes_pix;


tabela,total_registros
silver.transacoes_pix,598924


## Schema e tipos de dados

A validação do schema e dos tipos de dados teve como objetivo confirmar que os campos foram corretamente definidos após o tratamento na camada Silver, especialmente aqueles utilizados em cálculos e agregações. Essa análise assegura que valores numéricos, temporais e categóricos estejam representados de forma adequada, reduzindo o risco de erros nas análises e garantindo consistência técnica no modelo analítico.

In [0]:
DESCRIBE TABLE silver.transacoes_pix;


col_name,data_type,comment
AnoMes,int,null
ano,int,null
mes,int,null
pag_tipo_pessoa,string,null
rec_tipo_pessoa,string,null
pag_regiao,string,null
rec_regiao,string,null
pag_faixa_etaria,string,null
rec_faixa_etaria,string,null
valor_reais,"decimal(18,2)",null


## Análise de valores nulos por coluna

A análise de valores nulos foi realizada para avaliar a completude dos dados, identificando possíveis ausências de informação em atributos relevantes para as análises. A presença de valores nulos em campos críticos pode comprometer a interpretação dos resultados, tornando essa verificação essencial para garantir a confiabilidade das métricas calculadas.

In [0]:
SELECT
  SUM(CASE WHEN AnoMes IS NULL THEN 1 ELSE 0 END) AS AnoMes_nulo,
  SUM(CASE WHEN ano IS NULL THEN 1 ELSE 0 END) AS ano_nulo,
  SUM(CASE WHEN mes IS NULL THEN 1 ELSE 0 END) AS mes_nulo,
  SUM(CASE WHEN pag_regiao IS NULL THEN 1 ELSE 0 END) AS pag_regiao_nulo,
  SUM(CASE WHEN rec_regiao IS NULL THEN 1 ELSE 0 END) AS rec_regiao_nulo,
  SUM(CASE WHEN pag_tipo_pessoa IS NULL THEN 1 ELSE 0 END) AS pag_tipo_pessoa_nulo,
  SUM(CASE WHEN rec_tipo_pessoa IS NULL THEN 1 ELSE 0 END) AS rec_tipo_pessoa_nulo,
  SUM(CASE WHEN pag_faixa_etaria IS NULL THEN 1 ELSE 0 END) AS pag_faixa_etaria_nulo,
  SUM(CASE WHEN rec_faixa_etaria IS NULL THEN 1 ELSE 0 END) AS rec_faixa_etaria_nulo,
  SUM(CASE WHEN valor_reais IS NULL THEN 1 ELSE 0 END) AS valor_reais_nulo,
  SUM(CASE WHEN quantidade_total IS NULL THEN 1 ELSE 0 END) AS quantidade_total_nulo
FROM silver.transacoes_pix;


AnoMes_nulo,ano_nulo,mes_nulo,pag_regiao_nulo,rec_regiao_nulo,pag_tipo_pessoa_nulo,rec_tipo_pessoa_nulo,pag_faixa_etaria_nulo,rec_faixa_etaria_nulo,valor_reais_nulo,quantidade_total_nulo
0,0,0,0,0,0,0,0,0,0,0


## Análise de domínio dos dados numéricos

A análise de domínio dos dados numéricos teve como objetivo verificar se os valores financeiros e as quantidades de transações se encontram dentro de intervalos esperados e coerentes com o contexto do negócio. Essa verificação permite identificar valores inconsistentes ou atípicos que poderiam indicar falhas no processo de carga ou problemas na fonte de dados.

In [0]:
SELECT
  MIN(valor_reais) AS valor_minimo,
  MAX(valor_reais) AS valor_maximo,
  AVG(valor_reais) AS valor_medio
FROM silver.transacoes_pix;


valor_minimo,valor_maximo,valor_medio
0.01,554621985240.43,128864635.189971


## Quantidade de transações

In [0]:
SELECT
  MIN(quantidade_total) AS quantidade_minima,
  MAX(quantidade_total) AS quantidade_maxima,
  AVG(quantidade_total) AS quantidade_media
FROM silver.transacoes_pix;


quantidade_minima,quantidade_maxima,quantidade_media
1,228002349,309255.2121771711


## Análise de distribuição dos dados (quartis)

A análise de distribuição dos dados, por meio de quartis, foi utilizada para compreender a dispersão e o comportamento estatístico dos valores financeiros. Essa verificação auxilia na identificação de assimetrias e concentrações nos dados, oferecendo contexto adicional para a interpretação de médias e totais apresentados nas análises de negócio.

In [0]:
SELECT
  percentile_approx(valor_reais, 0.25) AS q1,
  percentile_approx(valor_reais, 0.50) AS mediana,
  percentile_approx(valor_reais, 0.75) AS q3
FROM silver.transacoes_pix;


q1,mediana,q3
6927.09,152820.11,3247879.32


## Análise de duplicidade de registros

A análise de duplicidade de registros foi aplicada para identificar possíveis repetições indevidas de dados que poderiam resultar em dupla contagem nas análises agregadas. Essa verificação é importante para garantir a unicidade lógica dos registros e assegurar que os resultados analíticos reflitam corretamente o volume real de transações.

In [0]:
SELECT
  AnoMes,
  pag_regiao,
  rec_regiao,
  pag_tipo_pessoa,
  rec_tipo_pessoa,
  pag_faixa_etaria,
  rec_faixa_etaria,
  COUNT(*) AS total_registros
FROM silver.transacoes_pix
GROUP BY
  AnoMes,
  pag_regiao,
  rec_regiao,
  pag_tipo_pessoa,
  rec_tipo_pessoa,
  pag_faixa_etaria,
  rec_faixa_etaria
HAVING COUNT(*) > 1;


AnoMes,pag_regiao,rec_regiao,pag_tipo_pessoa,rec_tipo_pessoa,pag_faixa_etaria,rec_faixa_etaria,total_registros
202211,NORTE,NORDESTE,PF,PF,20-29,20-29,6
202303,NORTE,NORTE,PF,PJ,60+,Nao se aplica,11
202301,SUDESTE,Nao informado,PJ,PF,Nao se aplica,30-39,5
202401,SUL,NORDESTE,PJ,PJ,Nao se aplica,Nao se aplica,12
202408,NORDESTE,SUL,PF,PF,50-59,20-29,5
202307,NORTE,Nao informado,PF,PF,50-59,40-49,5
202210,SUL,SUDESTE,PF,PF,50-59,30-39,5
202211,CENTRO-OESTE,NORTE,PF,PF,50-59,30-39,5
202211,CENTRO-OESTE,SUDESTE,PJ,PJ,Nao se aplica,Nao se aplica,14
202210,NORTE,NORTE,PF,PJ,60+,Nao se aplica,12


## Análise de integridade referencial

A análise de integridade referencial teve como objetivo verificar a consistência entre os dados que compõem o modelo estrela, assegurando que os registros da tabela fato estejam corretamente associados aos valores das dimensões. Essa validação é essencial para garantir a coerência do modelo dimensional e a confiabilidade das consultas analíticas realizadas na camada Gold.

### Regiões inválidas

In [0]:
SELECT DISTINCT pag_regiao
FROM silver.transacoes_pix
WHERE pag_regiao IS NULL;


pag_regiao


### Tipo de pessoa inválido

In [0]:
SELECT DISTINCT pag_tipo_pessoa
FROM silver.transacoes_pix
WHERE pag_tipo_pessoa NOT IN ('PF', 'PJ');


pag_tipo_pessoa
Nao disponivel


### Faixa etária inválida

In [0]:
SELECT DISTINCT pag_faixa_etaria
FROM silver.transacoes_pix
WHERE pag_faixa_etaria NOT IN ('0-19','20-29','30-39','40-49','50-59','60+','Nao se aplica','NAO_INFORMADO');


pag_faixa_etaria


## Análise de consistência temporal

A análise de consistência temporal foi realizada para verificar se os registros seguem uma sequência temporal válida e coerente, sem a presença de períodos inválidos ou inconsistentes. Essa verificação é especialmente importante em análises baseadas em séries temporais, garantindo que comparações entre meses e períodos sejam realizadas de forma correta.

In [0]:
SELECT DISTINCT AnoMes
FROM silver.transacoes_pix
ORDER BY AnoMes;


AnoMes
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108


## Análise de cardinalidade das dimensões

A análise de cardinalidade das dimensões teve como objetivo avaliar a quantidade de valores distintos nos principais atributos categóricos, assegurando que as dimensões apresentem granularidade adequada ao modelo estrela. Essa verificação contribui para validar a escolha do modelo dimensional e reforça sua eficiência para análises analíticas e segmentações.

In [0]:
SELECT COUNT(DISTINCT pag_regiao) AS total_regioes
FROM silver.transacoes_pix;


total_regioes
6


# Considerações

A análise de qualidade dos dados foi realizada na camada Silver, onde os dados já se encontram tratados e padronizados. Foram avaliados volume, schema, completude, domínio dos dados numéricos, distribuição estatística, duplicidade e consistência dos atributos categóricos, assegurando a confiabilidade das informações antes da modelagem dimensional na camada Gold.